In [1]:
import cv2
import os
import tensorflow as tf
import glob
import numpy as np

In [2]:
TRAIN_IMAGES_PATH = './datasets/images/train'
TEST_IMAGES_PATH = './datasets/images/test'
MODEL_OUTPUT_PATH = './datasets/output/'

In [3]:
def read_captcha_images(image_path):
    if not os.path.exists(image_path):
        raise ValueError('image path is not exist')
        
    images = []
    labels = []
    images_path = os.path.join(image_path, '*.jpg')
    count = 0
    for image_file in glob.glob(images_path):
        count += 1
        if count % 5000 == 0:
            print('Load {} images.'.format(count))
        image = cv2.imread(image_file)
        
        #image = image[:, :, (2, 1, 0)] # change channel
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        label = int(image_file.split('_')[-1].split('.')[0])
        images.append(image)
        labels.append(label)
    images = np.array(images)
    labels = np.array(labels)
    return images, labels

In [4]:
def next_batch_set(images, labels, batch_size=128):
    """Generate a batch training data.
    
    Args:
        images: A 4-D array representing the training images.
        labels: A 1-D array representing the classes of images.
        batch_size: An integer.
        
    Return:
        batch_images: A batch of images.
        batch_labels: A batch of labels.
    """
    indices = np.random.choice(len(images), batch_size)
    batch_images = images[indices]
    batch_labels = labels[indices]
    return batch_images, batch_labels

In [5]:
#test_images, test_targets = read_captcha_images(FLAGS.test_images_path)
train_images, train_targets = read_captcha_images(TRAIN_IMAGES_PATH)
test_images, test_targets = next_batch_set(train_images, train_targets)

Load 5000 images.
Load 10000 images.
Load 15000 images.
Load 20000 images.
Load 25000 images.
Load 30000 images.
Load 35000 images.
Load 40000 images.


In [6]:
def load_model():
    with tf.Session() as sess:
        saver = tf.train.import_meta_graph('./model/captcha_cnn_model_data.meta')
        saver.restore(sess, tf.train.latest_checkpoint("./model/"))
        graph = tf.get_default_graph()
        inputs = graph.get_tensor_by_name("inputs:0")
        #drop_prob = graph.get_tensor_by_name('dropout11/Placeholder:0') 
        feed_dict ={inputs:test_images}

        #Now, access the op that you want to run. 
        op_to_restore = graph.get_tensor_by_name("accuracy/predict:0")

        return sess.run(op_to_restore,feed_dict)

In [7]:
classes = load_model()

INFO:tensorflow:Restoring parameters from ./model/captcha_cnn_model_data


In [8]:
print (np.mean(np.equal(classes, test_targets)))

0.0625


In [9]:
classes

array([5, 8, 2, 0, 4, 3, 0, 0, 5, 0, 0, 0, 0, 0, 3, 5, 0, 2, 9, 0, 0, 0,
       5, 9, 8, 0, 3, 8, 5, 2, 6, 5, 0, 9, 0, 9, 0, 9, 5, 0, 8, 9, 8, 9,
       8, 9, 9, 9, 5, 2, 8, 5, 5, 9, 6, 3, 0, 8, 9, 6, 9, 3, 3, 0, 2, 8,
       9, 8, 3, 0, 2, 0, 9, 5, 3, 8, 8, 0, 5, 5, 0, 0, 9, 4, 9, 0, 0, 5,
       0, 0, 9, 8, 0, 6, 9, 3, 4, 8, 0, 5, 6, 3, 4, 5, 0, 0, 0, 0, 9, 0,
       8, 0, 6, 9, 0, 0, 0, 0, 5, 0, 8, 0, 8, 9, 0, 4, 0, 5])

In [10]:
test_targets

array([2, 9, 0, 6, 1, 3, 8, 2, 8, 9, 8, 7, 4, 3, 4, 8, 7, 3, 4, 3, 6, 1,
       4, 3, 1, 1, 9, 2, 9, 0, 0, 4, 5, 1, 2, 8, 8, 6, 2, 3, 1, 3, 5, 5,
       8, 1, 9, 5, 3, 3, 1, 0, 9, 9, 2, 6, 8, 7, 4, 9, 1, 5, 6, 1, 6, 1,
       7, 9, 5, 9, 4, 5, 8, 7, 9, 1, 5, 1, 3, 6, 3, 3, 3, 0, 5, 5, 3, 2,
       6, 1, 4, 2, 7, 9, 4, 5, 3, 9, 2, 4, 6, 9, 6, 5, 4, 3, 1, 3, 7, 5,
       1, 8, 1, 9, 5, 0, 4, 1, 9, 5, 9, 3, 6, 1, 7, 7, 7, 7])